In [4]:
# # 下载缺失的库，如果没有安装，会自动进行安装
# import sys
# import subprocess

# def install_if_missing(pkg):
#     try:
#         __import__(pkg)
#     except ImportError:
#         print(f"🔄 Installing missing package: {pkg}")
#         subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

# # 检查并安装 notebook所需的库
# for pip_pkg, mod_name in [
#     ("google-genai", "google.genai"),
#     ("langchain", "langchain"),
#     ("langchain-google-genai", "langchain_google_genai"),
#     ("python-dotenv", "dotenv")
# ]:
#     try:
#         __import__(mod_name)
#     except ImportError:
#         subprocess.check_call([sys.executable, "-m", "pip", "install", pip_pkg])


In [5]:
# pip install -U google-genai langchain langchain-google-genai python-dotenv

import os
import json
from dotenv import load_dotenv
import google.genai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage

load_dotenv()
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

# 缓存文件路径
CACHE_FILE = "file_cache.json"

# 示例 PDF 路径
question_pdf_path = ".example/paper.pdf"
solution_pdf_path = ".example/solution.pdf"

def load_cache():
    """读取缓存 JSON"""
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return {}

def save_cache(cache):
    """保存缓存 JSON"""
    with open(CACHE_FILE, "w", encoding="utf-8") as f:
        json.dump(cache, f, indent=2, ensure_ascii=False)

def upload_if_needed(path, cache_key):
    cache = load_cache()
    if cache_key in cache:
        print(f"✅ 已缓存: {cache_key} → {cache[cache_key]}")
        return cache[cache_key]

    print(f"⏫ 正在上传 {path} ...")
    uploaded = client.files.upload(file=path)   # ← 关键：用 file=path
    # 也可：uploaded = client.files.upload(file=open(path, "rb"))
    cache[cache_key] = uploaded.uri
    save_cache(cache)
    print(f"✅ 上传完成: {cache_key} → {uploaded.uri}")
    return uploaded.uri


# === 1. 确保文件已上传一次 ===
q_uri = upload_if_needed(question_pdf_path, "paper_pdf_uri")
s_uri = upload_if_needed(solution_pdf_path, "solution_pdf_uri")

# === 2. 初始化 Gemini 模型 ===
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

agent = create_agent(
    model=llm,
    tools=[],
    system_prompt="You are an Australian high school teacher.",
)

# === 3. 构造消息并执行 ===
question = HumanMessage(content=[
    {"type": "text", "text":
        "You are an Australian high school teacher.\n"
        "Please ONLY summarize the content of the third page of the solution PDF. Do NOT scan the whole PDF. "
    },
    {"type": "media", "mime_type": "application/pdf", "file_uri": q_uri},
    {"type": "media", "mime_type": "application/pdf", "file_uri": s_uri},
])


✅ 已缓存: paper_pdf_uri → https://generativelanguage.googleapis.com/v1beta/files/miz7rnjted0s
✅ 已缓存: solution_pdf_uri → https://generativelanguage.googleapis.com/v1beta/files/f7mbxctr2i1e


In [6]:
for step in agent.stream({"messages": [question]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

[{'type': 'text', 'text': 'You are an Australian high school teacher.\nPlease ONLY summarize the content of the third page of the solution PDF. Do NOT scan the whole PDF. '}, {'type': 'media', 'mime_type': 'application/pdf', 'file_uri': 'https://generativelanguage.googleapis.com/v1beta/files/miz7rnjted0s'}, {'type': 'media', 'mime_type': 'application/pdf', 'file_uri': 'https://generativelanguage.googleapis.com/v1beta/files/f7mbxctr2i1e'}]
================================== Ai Message ==================================

The third page of the solution PDF contains the solution to Question 11 (b) from the Mathematics Extension 1 exam.

Question 11 (b) asks to find the values of two expressions related to the roots (α, β, γ) of the cubic equation -2x³ – 4x² – 9x + 6 = 0.

1.  **Part (i): Find the value of 1/α + 1/β + 1/γ.**
    *   The solution uses the relationships between the roots and coefficients of a cu